In [1]:
from functools import partial
import time
import os
import fire
import tqdm
import json
import random
import datasets
import numpy as np
import argparse
from distutils.util import strtobool

from sacrebleu.metrics import BLEU
from transformers import AutoTokenizer
from tokenizers import ByteLevelBPETokenizer

import minitorch
from minitorch import DecoderLM
from minitorch.cuda_kernel_ops import CudaKernelOps


def get_dataset(dataset_name, model_max_length):
    """
    Obtrain IWSLT (de-en) dataset.
    """
    dataset = {
        split: datasets.load_dataset(dataset_name, split=split)['translation']
        for split in ['train', 'validation', 'test']
    }
    src_key, tgt_key = 'de', 'en'

    dataset = {
        split: [
            example for example in dataset[split]
            if len(example[src_key].split()) + len(example[tgt_key].split()) < model_max_length
        ] for split in dataset.keys()
    }

    dataset['test'] = dataset['test'][:100]             # 6750

    print(json.dumps(
        {'data_size': {split: len(dataset[split]) for split in dataset.keys()}},
        indent=4))

    return dataset, src_key, tgt_key


def get_tokenizer(examples, vocab_size, src_key, tgt_key, workdir):
    """
    Trains a tokenizer on the provided dataset examples and saves the tokenizer configuration.

    Parameters:
    - examples: The dataset examples used for training the tokenizer.
    - vocab_size: The desired vocabulary size for the tokenizer.
    - src_key: The key used to access the source text within the dataset examples.
    - tgt_key: The key used to access the target text within the dataset examples.
    - workdir: The directory where the tokenizer should be saved.

    Returns:
    - tokenizer: The trained tokenizer with special tokens,
        e.g., ("<eos_de>", "<eos_en>", "<pad>") if src_key and tgt_key are "de" and "en", respectively.
    """
    tokenizer = ByteLevelBPETokenizer()

    # Customized training
    tokenizer.train_from_iterator(
        [[example[src_key], example[tgt_key]] for example in examples],
        vocab_size=vocab_size,
        special_tokens=[f'<eos_{src_key}>', f'<eos_{tgt_key}>', '<pad>'])

    tokenizer.save(f'{workdir}/tokenizer.json')
    json.dump({'model_type': 'gpt2'}, open(f'{workdir}/config.json', 'w'))

    tokenizer = AutoTokenizer.from_pretrained(
        workdir,
        eos_token=None,
        bos_token=None,
        pad_token=None,
        unk_token=None)

    return tokenizer


def collate_batch(
        examples, src_key, tgt_key, tokenizer, model_max_length, backend):
    """
    Prepares a batch of examples for model training or evaluation by tokenizing and padding them.

    Parameters:
    - examples: A list of examples to be processed.
    - src_key: The key for accessing source texts in the examples.
    - tgt_key: The key for accessing target texts in the examples.
    - tokenizer: The tokenizer to be used for encoding the texts.
    - model_max_length: The maximum sequence length the model can handle.
    - backend: The backend of minitorch tensors.

    Returns:
    - A dictionary containing keys: 'input_ids', 'labels', 'label_token_weights',
        each indicates a minitorch tensor with shape (len(examples), model_max_length).

    Notes:
    ["input_ids"] for every example in the DE-EN translation, the "input_ids" will be:
        <de_token_ids> + <de_eos_id> + <en_token_ids> + <en_eos_id> + <pad_ids>
    where the pad_ids makes the length of input_ids to be model_max_length.

    ["labels"]: the next tokens to be predicted, which will be used in the cross-entropy
    loss function, e.g., for an example tokenized as [a, b, c, d], "input_ids" and "labels" 
    can be [a, b, c] and [b, c, d], respectively.

    ["label_token_weights"] The 'label_token_weights' are used to differentiate
    calculation purposes. (the MLE loss is computed on target tokens only.)
    between the source (weight = 0) and target (weight = 1) tokens for loss
    """
    token_ids, tgt_token_mask = [], []
    max_length = model_max_length
    pad_token_id = tokenizer.vocab['<pad>']
    for example in examples:
        token_ids_src = tokenizer(
            f'{example[src_key]}<eos_{src_key}>')['input_ids']
        token_ids_tgt = tokenizer(
            f'{example[tgt_key]}<eos_{tgt_key}>')['input_ids']

        example_token_ids = token_ids_src + token_ids_tgt
        example_tgt_token_mask = (
                [0] * len(token_ids_src) + [1] * len(token_ids_tgt))
        example_token_ids = example_token_ids[:max_length]
        example_tgt_token_mask = example_tgt_token_mask[:max_length]
        pad_ids = [pad_token_id] * (max_length - len(example_token_ids))

        token_ids.append(example_token_ids + pad_ids)
        tgt_token_mask.append(example_tgt_token_mask + [0] * len(pad_ids))

    # TODO: make examples in a 1d list, provide shape to initialize minitorch.Tensor
    token_ids = np.array(token_ids)
    tgt_token_mask = np.array(tgt_token_mask)

    input_ids = token_ids[:, :-1]
    labels    = token_ids[:, 1:]
    label_token_weights = tgt_token_mask[:, 1:]

    input_ids = minitorch.tensor_from_numpy(input_ids, backend=backend)
    labels    = minitorch.tensor_from_numpy(labels, backend=backend)
    label_token_weights = minitorch.tensor_from_numpy(label_token_weights, backend=backend)
    
    # input_ids = token_ids[:, :-1].tolist()
    # labels    = token_ids[:, 1:].tolist()
    # label_token_weights = tgt_token_mask[:, 1:].tolist()

    # input_ids = minitorch.tensor(input_ids, backend=backend)
    # labels    = minitorch.tensor(labels, backend=backend)
    # label_token_weights = minitorch.tensor(label_token_weights, backend=backend)

    return {
        'input_ids': input_ids,
        'labels': labels,
        'label_token_weights': label_token_weights
    }



def loss_fn(batch, model):
    """
    The MLE loss for a batch.

    Parameters:
    - batch: The result of collate_fn, a dict with "input_ids", "labels", and "label_token_weights".
    - model: The model to be trained.

    Returns:
    - A scalar loss value for this batch, averaged across all target tokens.
    """

    idx = batch['input_ids']
    idx.requires_grad_(True)
    # print("getting into loss_fn")
    logits = model(idx=idx)
    # print("finish prediction")
    bs, l, c = logits.shape
    logits = logits.view(bs * l, c)
    targets = batch['labels'].view(bs * l)
    label_token_weights = batch['label_token_weights'].view(bs * l)

    targets.requires_grad_(True)
    # print("start calculating loss")
    # import pdb
    # pdb.set_trace()
    loss = minitorch.nn.softmax_loss(
        logits=logits,
        target=targets
    )

    return ((loss * label_token_weights).sum() / label_token_weights.sum())


def train(model, optimizer, examples, n_samples, collate_fn, batch_size, desc):
    model.train()
    random.shuffle(examples)
    examples = examples[:n_samples]

    for i in (prog_bar := tqdm.trange(
            0, len(examples), batch_size, desc=f'Training ({desc})')):
        
        batch = collate_fn(examples=examples[i:i + batch_size])
        
        t0 = time.time()
        optimizer.zero_grad()
        loss = loss_fn(batch=batch, model=model)
        t1 = time.time()

        loss.backward()
        t2 = time.time()

        optimizer.step()
        t3 = time.time()

        # print(f"Forward: {t1 - t0}")
        # print(f"Backward: {t2 - t1}")
        # print(f"Opt.step: {t3 - t2}")

        batch_time = time.time() - t0
        prog_bar.set_postfix(
            tokens_per_sec=np.prod(batch['input_ids'].shape) / batch_time,
            loss=loss.item(),
            lr=optimizer.lr)


def parse_args():
    def str2bool(x):
        return bool(strtobool(x))
        
    parser = argparse.ArgumentParser()
    parser.add_argument('--use-fused-kernel', type=str2bool, default=False)
    return parser.parse_args()



In [2]:


dataset_name='iwslt14-de-en-preprocess'
model_max_length=40
n_epochs=1
batch_size=128
learning_rate=0.02
samples_per_epoch=128
n_vocab=10000
n_embd=256
seed=11111         
np.random.seed(seed)
random.seed(seed)

workdir = f'./workdir_vocab{n_vocab}_lr{learning_rate}_embd{n_embd}'
os.makedirs(workdir, exist_ok=True)

backend = minitorch.TensorBackend(CudaKernelOps)

config = {
    'n_vocab'     : n_vocab,  # vocab_size
    'n_embd'      : n_embd,   # n_embed
    'n_head'      : 8,    # n_head
    'n_positions' : model_max_length,  # n_ctx == n_positions
    # 'n_layer'     : 4,    # n_layer
    'p_dropout'   : 0.1,  # x_pdrop
    'ln_eps'      : 1e-5, # layer_norm_epsilon
    'backend'     : backend,
    'use_fused_kernel': True
}

model = DecoderLM(**config)
optimizer = minitorch.Adam(model.parameters(), lr=learning_rate)

dataset, src_key, tgt_key = get_dataset(
    dataset_name=dataset_name, model_max_length=model_max_length)

tokenizer = get_tokenizer(
    examples=dataset['train'],
    vocab_size=config['n_vocab'],
    src_key=src_key,
    tgt_key=tgt_key,
    workdir=workdir)

collate_fn = partial(
    collate_batch,
    src_key=src_key,
    tgt_key=tgt_key,
    tokenizer=tokenizer,
    model_max_length=model_max_length,
    backend=backend)


examples=dataset['train']
n_samples=samples_per_epoch
model.train()
random.shuffle(examples)
examples = examples[:n_samples]

batch = collate_fn(examples=examples[0:0 + batch_size])

optimizer.zero_grad()
# myloss = loss_fn(batch=batch, model=model)

idx = batch['input_ids']
idx.requires_grad_(True)
# print("getting into loss_fn")
logits = model(idx=idx)
# print("finish prediction")
bs, l, c = logits.shape
logits = logits.view(bs * l, c)
targets = batch['labels'].view(bs * l)
label_token_weights = batch['label_token_weights'].view(bs * l)

targets.requires_grad_(True)
# print("start calculating loss")
# import pdb
# pdb.set_trace()
loss = minitorch.nn.softmax_loss(
    logits=logits,
    target=targets
)

myloss= ((loss * label_token_weights).sum() / label_token_weights.sum())


myloss.backward()
optimizer.step()

Using the latest cached version of the module from /home/alien/.cache/huggingface/modules/datasets_modules/datasets/iwslt14-de-en-preprocess/c16e61361ef4d92649321b04532f0d57c5a783ad7fd32afe4a9649a6b6107f8a (last modified on Mon May  5 21:12:05 2025) since it couldn't be found locally at iwslt14-de-en-preprocess., or remotely on the Hugging Face Hub.
Reusing dataset iwslt14-de-en-preprocess (/home/alien/.cache/huggingface/datasets/iwslt14-de-en-preprocess/de-en/1.0.0/c16e61361ef4d92649321b04532f0d57c5a783ad7fd32afe4a9649a6b6107f8a)
Using the latest cached version of the module from /home/alien/.cache/huggingface/modules/datasets_modules/datasets/iwslt14-de-en-preprocess/c16e61361ef4d92649321b04532f0d57c5a783ad7fd32afe4a9649a6b6107f8a (last modified on Mon May  5 21:12:05 2025) since it couldn't be found locally at iwslt14-de-en-preprocess., or remotely on the Hugging Face Hub.
Reusing dataset iwslt14-de-en-preprocess (/home/alien/.cache/huggingface/datasets/iwslt14-de-en-preprocess/de-e

{
    "data_size": {
        "train": 97976,
        "validation": 4512,
        "test": 100
    }
}





In [3]:
myloss


[9.345477]

In [9]:
logits.to_numpy()[0]

array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [ ]:
logits

In [6]:
unfusedmodel = model

In [11]:
unfusedmodel.ln.weights


[0.998000 0.998000 0.998001 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 1.002000 0.998000 0.998000 1.002000 0.998000 1.002000 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 1.002000 0.998000 1.002000 0.998000 0.998000 0.998000 0.998001 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 1.001997 0.998001 0.998000 1.002000 0.998000 0.998000 0.998000 0.998000 0.998000 1.002000 1.002000 0.998000 0.998000 0.998000 0.998001 0.998002 0.998000 1.002000 0.998000 1.001999 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 0.998001 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 1.002000 0.998000 0.998000 0.998000 1.002000 0.998000 1.002000 1.002000 0.998000 1.002000 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 0.998000 1.002000 0.998000 1.002000 0.998000 0.998000 0.998000 1.002000 0.998000 1.002000 0.998000 0.998000 1.002000 0.998000 1.002000 1.002000 0.998000 1.001987 1.002000 0.998000

In [15]:
model.ln.weights.value


[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan na